https://arokem.github.io/python-matlab-bridge/

In [1]:
from pymatbridge import Octave, Matlab

In [2]:
mat = Octave() # change to Matlab if you want, you can also provide path to matlab/octave binary

In [3]:
mat.start()

Starting Octave on ZMQ socket ipc:///tmp/pymatbridge-c4b17f3d-e37e-46c6-8256-38b8bc0b28cd
Send 'exit' command to kill the server
.Octave started and connected!


In [4]:
result = mat.run_code('a=1;')
print('success:', result['success'])

success: True


In [5]:
a = mat.get_variable('a')
print(a)

1


In [8]:
%%file foo.m
function output = foo(args)
    x = args.x;
    y = args.y;
    output = x + y;
end

Overwriting foo.m


In [10]:
result = mat.run_func('foo.m', dict(x=5, y=6))
print('success', result['success'])
print('result', result['result'])

success True
result 11


## Fibonacci

In [37]:
%%file fib.m
function f = fibonacci(args)
    n = args.n;
    f = zeros(n,1);
    f(1) = 1;
    f(2) = 1;
    for k = 3:n
        f(k) = f(k-1) + f(k-2);
    end
end

Overwriting fib.m


In [50]:
def matfib(n):
    result = mat.run_func('fib.m', dict(n=n))
    assert result['success']
    return result['result'].squeeze() # MATLAB returns 2D
matfib(5)

array([ 1.,  1.,  2.,  3.,  5.])

In [51]:
import numpy as np
def fib(n):
    f = np.zeros(n)
    f[0] = 1
    f[1] = 1
    for k in range(2, n):
        f[k] = f[k-1] + f[k-2]
    return f
fib(5)

array([ 1.,  1.,  2.,  3.,  5.])

In [52]:
%timeit fib(50)
%timeit matfib(50)

100000 loops, best of 3: 16.9 µs per loop
10 loops, best of 3: 135 ms per loop


In [59]:
x = np.arange(100).reshape(10, 10)
mat.set_variable('x', x)
mat.run_code('y=x*2')
y = mat.get_variable('y')
y[:3, :3]

array([[  0.,   2.,   4.],
       [ 20.,  22.,  24.],
       [ 40.,  42.,  44.]])

## Jupyter magic

In [15]:
# Octave
ip = get_ipython()
import pymatbridge
pymatbridge.load_ipython_extension(ip, matlab='octave')

# MATLAB
# %load_ext pymatbridge

Starting Octave on ZMQ socket ipc:///tmp/pymatbridge-74517818-e337-44c1-9490-8c86245a7fe1
Send 'exit' command to kill the server
.Octave started and connected!


In [16]:
%%matlab
a=1

a =  1


In [20]:
%%matlab
function [ result ] = fibonacci( n )

if n==0||n==1
    result = n;

else
    result = fibonacci(n-2)+fibonacci(n-1);
end
end

fibonacci(10)

ans =  55


In [21]:
def fibonacci(n):
    a = 1
    b = 1
    i = 2
    while i < n:
        a, b = b, a + b
        i+= 1
    return b